In [ ]:
from neomaril_codex.training import NeomarilTrainingClient

In [ ]:
client = NeomarilTrainingClient()
client

In [ ]:
training = client.create_training_experiment('Teste', 'Classification', 'External', group='datarisk')

In [ ]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
base_path = './samples/train/'

In [ ]:
df = pd.read_csv(base_path+"/dados.csv")

In [ ]:
X = df.drop(columns=['target'])
y = df[["target"]]

In [ ]:
pipe = make_pipeline(SimpleImputer(), LGBMClassifier(force_col_wise=True))

In [ ]:
with training.log_train('Teste 1', X, y) as logger:
    pipe.fit(X, y)
    logger.save_model(pipe)

    params = pipe.get_params()
    params.pop('steps')
    params.pop('simpleimputer')
    params.pop('lgbmclassifier')
    logger.save_params(params)

    model_output = pd.DataFrame({"pred": pipe.predict(X), "proba": pipe.predict_proba(X)[:,1]})
    logger.save_model_output(model_output)

    auc = cross_val_score(pipe, X, y, cv=5, scoring="roc_auc")
    f_score = cross_val_score(pipe, X, y, cv=5, scoring="f1")
    logger.save_metric(name='auc', value=auc.mean())
    logger.save_metric(name='f1_score', value=f_score.mean())

    logger.set_python_version('3.10')